# Application of Doc2Vec
on facebook posts from politicans in the German election 2017

In [380]:
from plotly import __version__
from plotly import offline as pyoff

In [381]:
pyoff.init_notebook_mode(connected=True)

In [3]:
import pandas as pd
import numpy as np
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import logging
import multiprocessing
import os
from collections import namedtuple

FORMAT = '%(asctime)s %(levelname)s %(message)s'
DATEFORMAT = '%Y-%m-%d %H:%M:%S'
logging.basicConfig(level=logging.INFO,
                    format=FORMAT,
                    datefmt=DATEFORMAT)

In [5]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(parent_dir, 'data')
models_dir = os.path.join(parent_dir, 'models')
print('working directory: ', os.getcwd())
print('data directory:    ', data_dir, )
print('models directory:  ', models_dir)

working directory:  /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/notebooks
data directory:     /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/data
models directory:   /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models


## loads the data

In [6]:
data = pd.read_pickle(os.path.join(data_dir, 'data_clean_4cols.pickle'))
data = data.drop(['id'], axis=1)

pd.set_option('max_colwidth', 1000)
data.sample(n=10)

,from_name,message,Partei_ABK
61170,Johannes Kahrs,"ihre erststimme. mein einsatz: der fernsehturm wird wiedereröffnet. noch weit vorher, am 1. april gibt es die chance, den telemichel zu besuchen - ich verlose eine führung mit anschließendem kaffeetrinken! jetzt mitmachen & teilen: www.kahrs.de/verlosung",SPD
167223,Kai Whittaker,Eindrücke vom heutigen Jugendgipfel in Rastatt #whittaker #merkel #cdu #btw17,CDU
74841,Jutta Krellmann,"Heute habe ich bei den Seniorinnen und Senioren der IG Metall in Alfeld auf dem Podium zum Thema ""Zusammen gegen soziale Kälte. Gerechte Politik für alle Generationen"" gesessen. Eine tolle Veranstaltung mit über 100 Gästen und guten Diskussionsbeiträgen. Danke für die Einladung und die tolle Organisation.",DIE LINKE
21528,Martin Diedenhofen,"Die Rhein-Zeitung beginnt mit der Berichterstattung zur Bundestagswahl. Ich kämpfe um die Erststimme und möchte die Menschen in unserem Wahlkreis überzeugen. Dafür bin ich tagtäglich unterwegs, dafür setzen wir uns mit einem super Team und vielen Unterstützern ein. Für das bald erscheinende Portrait habe ich mich mit der zuständigen Redakteurin auf dem Erpeler Sportplatz getroffen, dazu bald mehr! :) #TeamDiedenhofen #AufGehts #GemeinsamAnpacken",SPD
126675,Claudia Roth,Heute Zweitstimme Grün! Denn ohne Grün wird alles grau 💚,GRÜNE
140945,Uwe Schummer,"Zwar noch bei Regen, aber mit klasse Stimmung! Toller Einsatz in Dülken! #btw2017 #erfahrungschafftvertrauen #heimat /cp",CDU
108609,Cem Özdemir,"Liebe Freundinnen und Freunde,\n\nbis kommenden Freitag müssen alle Urwahl-Briefe eingegangen sein. Macht mit! Jede Stimme zählt.\n\nDie letzten Wochen haben gezeigt: Es geht um viel, und es wird ein harter Wahlkampf.\n\nIch stehe für das moderne und weltoffene Deutschland und ein solidarisches Europa. Es geht 2017 um den Erhalt des Miteinanders, des Zuhörens und des miteinander Redens. Wir dürfen uns den Zusammenhalt und unseren Zukunftsoptimismus nicht nehmen lassen!\n\nDas heißt: Klare Kante gegen alle, die unsere Freiheit bekämpfen. Gegen rechte Populisten, Nationalisten und Islamisten. Freiheit und Sicherheit sind kein Gegensatz. Wir brauchen beides!\n\nMit Euch will ich dafür sorgen, dass jedes Kind seine Träume verwirklichen kann und eine Chance hat, etwas aus seinem Leben machen zu können – egal woher es kommt.\n\nMit Euch streite ich für eine Wirtschaft, die mit Klimaschutz gute Arbeitsplätze schafft, und für eine Landwirtschaft, die unsere Natur, unsere Tiere und unser Wa...",GRÜNE
40105,Gregor Gysi,"Viele Menschen haben eine Hoffnung auf eine Sozialdemokratie, wie sie wirkte unter Willy Brandt. Wenn Martin Schulz nun, obwohl es eine andere Möglichkeit gäbe, eine Koalition mit der FDP oder der CDU einginge, machte er die SPD kaputt.",DIE LINKE
129135,Bernd Rützel,Ankunft in Himmelstadt.,SPD
177213,DIE LINKE Party,"Jedes Jahr zu Beginn der Sommerferien werden tausende Lehrer*innen entlassen. Nur um ein paar Gehälter einzusparen. Da die befristeten Verträge nicht mal für ein Jahr ausgelegt sind, können die meisten Lehrer*innen nicht mal Arbeitslosengeld beantragen. Nach den Ferien geht das ganze Spiel dann wieder von vorne los. Wir fordern feste Arbeitsverträge für alle Lehrer*innen! Der Lehrerberuf muss attraktiver werden – die Lehrer*innen haben es verdient!",DIE LINKE


##### gives doc2vec a challenge

In [365]:
any(data.message.str.count('FDP')>0)

False

In [7]:
# removes party mentions to test w
for char in ['SPD', 'spd', 'FDP', 'fdp', 'CDU' 'cdu', 'AfD' 'afd', 'AFD', 'Grüne', 'GRÜNE', 'Die Grünen', 'GRÜNEN', 'Linke', 'LINKE', 'CSU', 'csu', 'Die Linke', 'DIE LINKE',]:
        data.message = data.message.str.replace(char, '')

In [25]:
from nltk.tokenize.casual import TweetTokenizer

stopword_set = set(stopwords.words('german'))
tokenizer = TweetTokenizer
MessageDoc = namedtuple('MessageDoc', 'words tags')

alldocs = []  # Will hold all doacs in original order
for line_no, line in data.iterrows():
    #import pdb; pdb.set_trace()
    message = line.message.lower()
    words = [word for word in tokenizer().tokenize(message) if word not in stopword_set and word not in string.punctuation]
    tags = [str(line_no), line['from_name']] #, line['Partei_ABK']] # line_no needs to be converted as string to be included in tags 
    alldocs.append(MessageDoc(words, tags))

In [23]:
import string

message = data.message[7433]
words = [word for word in tokenizer().tokenize(message) if word not in stopword_set and word not in string.punctuation]

print('Post', '\n', message, '\n', '\n', 'Tokenization', '\n', words)

Post 
 Mein Ärger der Woche beim Sonntags-Stammtisch: die Abschiebung von Ahmad Pouya. Der afghanischen Künstler und Musiker lebte bis letzten Freitag 6 Jahre in Deutschland, spricht fließend Deutsch (und daneben fünf weitere Sprachen) und war vorbildlich in Deutschland integriert. Er hat als Dolmetscher gearbeitet und sollte eine Beratungsstelle für Geflüchtete leiten. Nun wurde er - trotz vieler Fürsprecher - eiskalt nach Afghanistan abgeschoben um ein Exempel zu statuieren. Noch immer bin ich fassungslos über dieses Vorgehen des Innenministers. 
 
 Tokenization 
 ['Mein', 'Ärger', 'Woche', 'beim', 'Sonntags-Stammtisch', 'Abschiebung', 'Ahmad', 'Pouya', 'Der', 'afghanischen', 'Künstler', 'Musiker', 'lebte', 'letzten', 'Freitag', '6', 'Jahre', 'Deutschland', 'spricht', 'fließend', 'Deutsch', 'daneben', 'fünf', 'weitere', 'Sprachen', 'vorbildlich', 'Deutschland', 'integriert', 'Er', 'Dolmetscher', 'gearbeitet', 'Beratungsstelle', 'Geflüchtete', 'leiten', 'Nun', 'wurde', 'trotz', 'viele

In [24]:
alldocs[7433]

MessageDoc(words=['ärger', 'woche', 'beim', 'sonntags-stammtisch', ':', 'abschiebung', 'ahmad', 'pouya', '.', 'afghanischen', 'künstler', 'musiker', 'lebte', 'letzten', 'freitag', '6', 'jahre', 'deutschland', ',', 'spricht', 'fließend', 'deutsch', '(', 'daneben', 'fünf', 'weitere', 'sprachen', ')', 'vorbildlich', 'deutschland', 'integriert', '.', 'dolmetscher', 'gearbeitet', 'beratungsstelle', 'geflüchtete', 'leiten', '.', 'wurde', '-', 'trotz', 'vieler', 'fürsprecher', '-', 'eiskalt', 'afghanistan', 'abgeschoben', 'exempel', 'statuieren', '.', 'immer', 'fassungslos', 'vorgehen', 'innenministers', '.'], tags=['7433', 'Margarete Bause'], split='train')

In [370]:
len(alldocs)

177307

In [371]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [462]:
alldocs[0:4]

[MessageDoc(words=['gerade', '9.000', 'anträge', 'kaufprämie', '#elektroautos', 'sechs', 'monaten', '–', 'weniger', 'hälfte', 'davon', 'privatpersonen', '!', 'eben', 'wunder', ',', 'staat', 'blindes', 'subventionieren', 'beschränkt', ',', 'anstatt', 'erforschung', 'speichermöglichkeiten', 'fördern', 'ladeinfrastruktur', 'auszubauen', '…', '#', '#btw17', '#hohenlohe', '#schwäbischhall', '#umwelt'], tags=['0', 'Valentin Abel'], split='train'),
 MessageDoc(words=['"', 'liberalen', 'stehen', 'individualistischen', 'freiheitsbegriff', ',', 'himmelweite', 'bürgerrechte', '.', 'zuversicht', 'hoffnung', 'näher', 'wut-und-blut-reden', '.', 'durchdenken', 'digitale', 'zukunft', ',', 'widmen', 'bildungspolitik', ',', 'suchen', 'diskussion', 'wirtschafts', '-', 'steuerpolitische', 'fragen', '.', 'themen', ',', 'land', 'wichtig', '[', '...', ']', '.', '"', 'hinzuzufügen', '!', ';-)'], tags=['1', 'Valentin Abel'], split='train'),
 MessageDoc(words=['bereit', '2017', '-', 'deutschland', 'bereit', 'zu

# doc2vec model

#### defines parameters

ran several models with changing parameters

In [487]:
from collections import OrderedDict

models = [
    #gensim.models.Doc2Vec(dm=0, size=60, negative=5, hs=0, min_count=5, workers=cores),
    gensim.models.Doc2Vec(dm=0, dbow_words=1, size=100, negative=5, hs=0, min_count=5, workers=cores),
    #gensim.models.Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, hs=0, min_count=2, iter=1, workers=cores),
    #gensim.models.Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=5, workers=cores),
]

#### builds vocabulary

In [488]:
# Speed up setup by sharing results of the 1st model's vocabulary scan
models[0].build_vocab(alldocs)  # PV-DM w/ concat requires one special NULL word so it serves as template
print(models[0])
for model in models[1:]:
    model.reset_from(models[0])
    print(model)

Doc2Vec(dbow+w,d100,n5,w5,mc5,s0.001,t4)


In [489]:
models_by_name = OrderedDict((str(model), model) for model in models[0:])

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
#models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([models[1], models[2]])
#models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([models[1], models[2]])

In [490]:
models_by_name

OrderedDict([('Doc2Vec(dbow+w,d100,n5,w5,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1ca1d8c128>)])

#### trains the models

In [491]:
epochs = 20

for name, train_model in models_by_name.items():
    train_model.train(alldocs, total_examples=len(alldocs), epochs=epochs, start_alpha=0.025, end_alpha=0.001)

#### saves the models

In [492]:
# saves the model
from datetime import datetime

count = 1
for name, trained_model in models_by_name.items():
    #import pdb; pdb.set_trace()
    name = name.replace('/', '')
    if count>len(models):
        break
    else:
        trained_model.save(os.path.join(models_dir, name + '_e' + str(epochs) + '.model'))
        count += 1
#logger.info('model saved')

## creates candidate specific data

In [586]:
data2 = pd.read_pickle(os.path.join(data_dir, 'data_clean_4cols.pickle'))
candidate_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
#                      .set_index('from_name')
                 )

In [574]:
candidate_data.tail(10)

,from_name,Partei_ABK
173042,Dr. Daniela De Ridder,SPD
173559,Björn Simon,CDU
173902,Waldemar Westermayer,CDU
173963,AfD Party,AfD
174481,CDU Party,CDU
174974,SPD Party,SPD
175488,CSU Party,CSU
176078,GRÜNE Party,GRÜNE
176425,FDP Party,FDP
176969,DIE LINKE Party,DIE LINKE


In [575]:
party_colors = {'AfD': 'rgb(0, 0, 153)',
                'DIE LINKE': 'rgb(204, 0, 102)',
                'GRÜNE': 'rgb(0, 153, 0)',
                'CSU': 'rgb(102, 178, 255)',
                'CDU': 'rgb(0, 0, 0)',
                'FDP': 'rgb(255, 255, 51)',
                'SPD': 'rgb(255, 0, 0)'}

candidate_data['color'] = candidate_data['Partei_ABK'].map(party_colors)

leaders = ['Sahra Wagenknecht',
'Dietmar Bartsch',
'Katrin Göring-Eckardt',
'Cem Özdemir',
'Martin Schulz',
'Angela Merkel',
'Joachim Herrmann',
'Alexander Gauland',
'Alice Weidel']

candidate_data['leader'] = data['from_name'].isin(leaders)

candidate_data['size'] = 5
candidate_data.loc[candidate_data['from_name'].str.contains('Party'),'size'] = 10
candidate_data.loc[candidate_data['leader'], 'size'] = 8

candidate_data['symbol'] = 100
candidate_data.loc[candidate_data['from_name'].str.contains('Party'),'symbol'] = 101
candidate_data.loc[candidate_data['leader'], 'symbol'] = 114

## plots model results

In [506]:
from sklearn.manifold import TSNE
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from time import time
import numpy as np
from sklearn import manifold
from sklearn.utils import check_random_state

In [27]:
model1_name = 'Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e1.model'
model2_name = 'Doc2Vec(dmm,d100,n5,w10,mc5,s0.001,t4)_e1.model' 
model3_name = 'Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e5.model'
model4_name = 'Doc2Vec(dmm,d100,n5,w10,mc5,s0.001,t4)_e5.model'
model5_name = 'Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e10.model'
model6_name = 'Doc2Vec(dmm,d100,n5,w10,mc5,s0.001,t4)_e10.model'
model7_name = 'Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e20.model'
model8_name = 'Doc2Vec(dmm,d100,n5,w10,mc5,s0.001,t4)_e20.model'

model1 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model1_name))
model2 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model2_name))
model3 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model3_name))
model4 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model4_name))
model5 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model5_name))
model6 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model6_name))
model7 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model7_name))
model8 = gensim.models.Doc2Vec.load(os.path.join(models_dir, model8_name))

2018-01-15 11:43:03 INFO loading Doc2Vec object from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e1.model
2018-01-15 11:43:06 INFO loading wv recursively from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e1.model.wv.* with mmap=None
2018-01-15 11:43:06 INFO setting ignored attribute syn0norm to None
2018-01-15 11:43:06 INFO loading docvecs recursively from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e1.model.docvecs.* with mmap=None
2018-01-15 11:43:06 INFO loading doctag_syn0 from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,

2018-01-15 11:43:25 INFO loading wv recursively from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e20.model.wv.* with mmap=None
2018-01-15 11:43:25 INFO setting ignored attribute syn0norm to None
2018-01-15 11:43:25 INFO loading docvecs recursively from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e20.model.docvecs.* with mmap=None
2018-01-15 11:43:25 INFO loading doctag_syn0 from /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models/Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)_e20.model.docvecs.doctag_syn0.npy with mmap=None
2018-01-15 11:43:26 INFO setting ignored attribute cum_table to None
2018-01-15 11:43:26 INFO loaded /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VW

In [534]:
fig1 = tools.make_subplots(rows=1, cols=2, subplot_titles=[model1_name, model2_name])
fig2 = tools.make_subplots(rows=1, cols=2, subplot_titles=[model3_name, model4_name])
fig3 = tools.make_subplots(rows=1, cols=2, subplot_titles=[model5_name, model6_name])
fig4 = tools.make_subplots(rows=1, cols=2, subplot_titles=[model7_name, model8_name])

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [535]:
# vector for model one
mask = [tag in candidate_data['from_name'].values for tag in model1.docvecs.offset2doctag]
candidate_vecs = model1.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne1 = tsne.fit_transform(X)

# vector for model two
mask = [tag in candidate_data['from_name'].values for tag in model2.docvecs.offset2doctag]
candidate_vecs = model2.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne2 = tsne.fit_transform(X)

# vector for model three
mask = [tag in candidate_data['from_name'].values for tag in model3.docvecs.offset2doctag]
candidate_vecs = model3.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne3 = tsne.fit_transform(X)

# vector for model four
mask = [tag in candidate_data['from_name'].values for tag in model4.docvecs.offset2doctag]
candidate_vecs = model4.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne4 = tsne.fit_transform(X)

# vector for model five
mask = [tag in candidate_data['from_name'].values for tag in model5.docvecs.offset2doctag]
candidate_vecs = model5.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne5 = tsne.fit_transform(X)

# vector for model six
mask = [tag in candidate_data['from_name'].values for tag in model6.docvecs.offset2doctag]
candidate_vecs = model6.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne6 = tsne.fit_transform(X)

# vector for model seven
mask = [tag in candidate_data['from_name'].values for tag in model7.docvecs.offset2doctag]
candidate_vecs = model7.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne7 = tsne.fit_transform(X)

# vector for model eight
mask = [tag in candidate_data['from_name'].values for tag in model8.docvecs.offset2doctag]
candidate_vecs = model8.docvecs.doctag_syn0[mask]

X = candidate_vecs
tsne = TSNE(n_components=2)
X_tsne8 = tsne.fit_transform(X)

In [536]:
trace = go.Scatter(x=X_tsne1[:, 0], y=X_tsne1[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig1.append_trace(trace, 1, 1)

trace = go.Scatter(x=X_tsne2[:, 0], y=X_tsne2[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig1.append_trace(trace, 1, 2)

trace = go.Scatter(x=X_tsne3[:, 0], y=X_tsne3[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig2.append_trace(trace, 1, 1)

trace = go.Scatter(x=X_tsne4[:, 0], y=X_tsne4[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig2.append_trace(trace, 1, 2)

trace = go.Scatter(x=X_tsne5[:, 0], y=X_tsne5[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig3.append_trace(trace, 1, 1)

trace = go.Scatter(x=X_tsne6[:, 0], y=X_tsne6[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig3.append_trace(trace, 1, 2)

trace = go.Scatter(x=X_tsne7[:, 0], y=X_tsne7[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig4.append_trace(trace, 1, 1)

trace = go.Scatter(x=X_tsne8[:, 0], y=X_tsne8[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
fig4.append_trace(trace, 1, 2)

In [537]:
fig1['layout'].update(title='Models with epochs=1', showlegend=False)
fig2['layout'].update(title='Models with epochs=5', showlegend=False)
fig3['layout'].update(title='Models with epochs=10', showlegend=False)
fig4['layout'].update(title='Models with epochs=20', showlegend=False)

In [538]:
pyoff.iplot(fig1, filename='tsnePlot1_epochs1')

In [539]:
pyoff.iplot(fig2, filename='tsnePlot1_epochs5')

In [540]:
pyoff.iplot(fig3, filename='tsnePlot1_epochs10<b')

In [541]:
pyoff.iplot(fig4, filename='tsnePlot1_epochs20')

## Assesses similarity between candidate and parties

In [595]:
# calculate similarity for all candidates and parties
candidate1_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate2_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate3_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate4_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate5_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate6_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate7_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )
candidate8_data = (data2.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
                                   )

# calculate similarity for all candidates and parties
for party in ['SPD Party', 'CDU Party', 'DIE LINKE Party', 'AfD Party', 'CSU Party', 'GRÜNE Party', 'FDP Party']:
    candidate1_data[party] = candidate1_data['from_name'].map(lambda candidate: model1.docvecs.similarity(candidate, party))
    candidate2_data[party] = candidate2_data['from_name'].map(lambda candidate: model2.docvecs.similarity(candidate, party))
    candidate3_data[party] = candidate3_data['from_name'].map(lambda candidate: model3.docvecs.similarity(candidate, party))
    candidate4_data[party] = candidate4_data['from_name'].map(lambda candidate: model4.docvecs.similarity(candidate, party))
    candidate5_data[party] = candidate5_data['from_name'].map(lambda candidate: model5.docvecs.similarity(candidate, party))
    candidate6_data[party] = candidate6_data['from_name'].map(lambda candidate: model6.docvecs.similarity(candidate, party))
    candidate7_data[party] = candidate6_data['from_name'].map(lambda candidate: model7.docvecs.similarity(candidate, party))
    candidate8_data[party] = candidate8_data['from_name'].map(lambda candidate: model8.docvecs.similarity(candidate, party))

# make a new column holding which party is most similar
candidate1_data['most similar'] = candidate1_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate2_data['most similar'] = candidate2_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate3_data['most similar'] = candidate3_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate4_data['most similar'] = candidate4_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate5_data['most similar'] = candidate5_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate6_data['most similar'] = candidate6_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate7_data['most similar'] = candidate7_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate8_data['most similar'] = candidate8_data.loc[:,'SPD Party':].idxmax(axis=1)

In [592]:
candidate1_data

,from_name,Partei_ABK,SPD Party,CDU Party,DIE LINKE Party,AfD Party,CSU Party,GRÜNE Party,FDP Party,most similar1
0,Valentin Abel,FDP,0.853713,0.840705,0.876193,0.689671,0.881301,0.873564,0.802148,CSU Party
93,Dr. Michael von Abercron,CDU,0.856357,0.841969,0.880810,0.694169,0.884045,0.877828,0.797343,CSU Party
168,Grigorios Aggelidis,FDP,0.851481,0.837296,0.877207,0.693393,0.879987,0.874015,0.796459,CSU Party
215,Diyar Agu,DIE LINKE,0.850998,0.837319,0.877093,0.693353,0.879959,0.873472,0.796737,CSU Party
269,Gökay Akbulut DIE LINKE,DIE LINKE,0.846841,0.834328,0.872528,0.689020,0.877065,0.870125,0.793912,CSU Party
284,Rolf Albach,FDP,0.856238,0.839513,0.880918,0.696258,0.883476,0.878167,0.796730,CSU Party
406,Stephan Albani MdB,CDU,0.873484,0.854754,0.891590,0.700813,0.899244,0.889851,0.801303,CSU Party
922,Katrin Albsteiger,CSU,0.860286,0.844604,0.883150,0.697017,0.887906,0.880902,0.798713,CSU Party
1017,Daniel Alff,SPD,0.841482,0.830635,0.870164,0.689109,0.873476,0.866953,0.792072,CSU Party
1033,Renata Alt,FDP,0.865150,0.849034,0.883869,0.696015,0.892483,0.881750,0.802743,CSU Party


In [597]:
most_similar_candidates = pd.crosstab(candidate7_data['Partei_ABK'], candidate7_data['most similar'])

In [598]:
most_similar_candidates

most similar,AfD Party,CDU Party,CSU Party,DIE LINKE Party,FDP Party,GRÜNE Party,SPD Party
Partei_ABK,,,,,,,
AfD,80,2,9,15,9,4,5
CDU,14,131,10,11,12,19,12
CSU,0,0,44,0,0,2,0
DIE LINKE,1,3,2,94,2,4,5
FDP,4,10,20,4,120,9,6
GRÜNE,3,2,11,11,9,55,4
SPD,5,13,34,24,13,25,136


In [600]:
# calculate average similarity of party candidates
candidate_data7 = (candidate7_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data7 = candidate_data7.stack().reset_index()

In [604]:
candidate_data7.sample(10)

,Partei_ABK,from_name,party_similarity,0
6456,SPD,Mathias Stein - Sozialdemokrat für den Deutschen Bundestag,DIE LINKE Party,0.393595
83,AfD,Beatrix von Storch,FDP Party,0.311864
6113,SPD,Jens Zimmermann,DIE LINKE Party,0.148520
1504,CDU,Johann Wadephul,FDP Party,0.285236
2481,CSU,"Hans-Peter Friedrich, CSU",AfD Party,0.081104
6110,SPD,Jens Niklaus,FDP Party,0.267488
2596,CSU,Stefan Kluge,FDP Party,0.303834
1207,CDU,Dr. Michael Meister,AfD Party,0.182261
6135,SPD,Johannes Kahrs,AfD Party,0.105027
684,AfD,Prof. Dr. Harald Weyel,GRÜNE Party,0.045084


In [605]:
average_similarity7 = pd.pivot_table(data=candidate_data7,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

In [643]:
import seaborn as sns
# inserting columns of zeros and ones to align the styling in the next step
# (otherwise, the colours would not mean the same in each row)
average_similarity_styled = average_similarity8.copy()
average_similarity_styled['0'] = 0.0
average_similarity_styled['1'] = 1.0
cm = sns.light_palette("blue", as_cmap=True)
average_similarity_styled = average_similarity_styled.style.background_gradient(cmap=cm, axis=1)
average_similarity_styled

party_similarity,AfD Party,CDU Party,CSU Party,DIE LINKE Party,FDP Party,GRÜNE Party,SPD Party,0,1
Partei_ABK,,,,,,,,,
AfD,0.53,0.22,0.32,0.22,0.25,0.21,0.19,0,1
CDU,0.16,0.42,0.3,0.13,0.27,0.24,0.25,0,1
CSU,0.17,0.26,0.6,0.09,0.29,0.26,0.23,0,1
DIE LINKE,0.16,0.15,0.15,0.54,0.2,0.36,0.28,0,1
FDP,0.17,0.26,0.31,0.23,0.53,0.3,0.24,0,1
GRÜNE,0.09,0.19,0.24,0.4,0.26,0.53,0.27,0,1
SPD,0.09,0.22,0.23,0.28,0.2,0.27,0.4,0,1


In [634]:
# computing the mean of diagonal elements
average_diagonal7 = np.trace(average_similarity7) / 7

# computing the mean of off-diagonal elements
average_off_diagonal7 = (average_similarity7.values.sum() - average_diagonal7 * 7) / 42

#the difference can be seen as a performance metric of the model 
average_diff7 = average_diagonal7 - average_off_diagonal7

In [28]:
print(model1_name, round(average_diff1, ndigits=4), '\n',
      model2_name, round(average_diff2, ndigits=4), '\n',
      model3_name, round(average_diff3, ndigits=4), '\n',
      model4_name, round(average_diff4, ndigits=4), '\n',
      model5_name, round(average_diff5, ndigits=4), '\n',
      model6_name, round(average_diff6, ndigits=4), '\n',
      model7_name, round(average_diff7, ndigits=4), '\n',
      model8_name, round(average_diff8, ndigits=4)
)

# code for other models is located in appendix

NameError: name 'average_diff1' is not defined

## Appendix

In [618]:
# calculate average similarity of party candidates
candidate_data1 = (candidate1_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data1 = candidate_data1.stack().reset_index()

average_similarity1 = pd.pivot_table(data=candidate_data1,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal1 = np.trace(average_similarity1) / 7

# computing the mean of off-diagonal elements
average_off_diagonal1 = (average_similarity1.values.sum() - average_diagonal1 * 7) / 42

#the difference can be seen as a performance metric of the model 
average_diff1 = average_diagonal1 - average_off_diagonal1

In [621]:
# calculate average similarity of party candidates
candidate_data2 = (candidate2_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data2 = candidate_data2.stack().reset_index()

average_similarity2 = pd.pivot_table(data=candidate_data2,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal2 = np.trace(average_similarity2) / 7

# computing the mean of off-diagonal elements
average_off_diagonal2 = (average_similarity2.values.sum() - average_diagonal2 * 7) / 42

#the difference can be seen as a performance metric of the model 
average_diff2 = average_diagonal2 - average_off_diagonal2

In [639]:
# calculate average similarity of party candidates
candidate_data3 = (candidate3_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data3 = candidate_data3.stack().reset_index()

average_similarity3 = pd.pivot_table(data=candidate_data3,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal3 = np.trace(average_similarity3) / 7

# computing the mean of off-diagonal elements
average_off_diagonal3 = (average_similarity3.values.sum() - average_diagonal3 * 7) / 43

#the difference can be seen as a performance metric of the model 
average_diff3 = average_diagonal3 - average_off_diagonal3

In [638]:
# calculate average similarity of party candidates
candidate_data4 = (candidate4_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data4 = candidate_data4.stack().reset_index()

average_similarity4 = pd.pivot_table(data=candidate_data4,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal4 = np.trace(average_similarity4) / 7

# computing the mean of off-diagonal elements
average_off_diagonal4 = (average_similarity4.values.sum() - average_diagonal4 * 7) / 44

#the difference can be seen as a performance metric of the model 
average_diff4 = average_diagonal4 - average_off_diagonal4

In [637]:
# calculate average similarity of party candidates
candidate_data5 = (candidate5_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data5 = candidate_data5.stack().reset_index()

average_similarity5 = pd.pivot_table(data=candidate_data5,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal5 = np.trace(average_similarity5) / 7

# computing the mean of off-diagonal elements
average_off_diagonal5 = (average_similarity5.values.sum() - average_diagonal5 * 7) / 55

#the difference can be seen as a performance metric of the model 
average_diff5 = average_diagonal5 - average_off_diagonal5

In [636]:
# calculate average similarity of party candidates
candidate_data6 = (candidate6_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data6 = candidate_data6.stack().reset_index()

average_similarity6 = pd.pivot_table(data=candidate_data6,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal6 = np.trace(average_similarity6) / 7

# computing the mean of off-diagonal elements
average_off_diagonal6 = (average_similarity6.values.sum() - average_diagonal6 * 7) / 66

#the difference can be seen as a performance metric of the model 
average_diff6 = average_diagonal6 - average_off_diagonal6

In [635]:
# calculate average similarity of party candidates
candidate_data8 = (candidate8_data
                   .set_index(['Partei_ABK', 'from_name'])
                   .drop(columns=['most similar'])
                   .rename_axis('party_similarity', axis='columns')
                   .sort_index()
                  )

candidate_data8 = candidate_data8.stack().reset_index()

average_similarity8 = pd.pivot_table(data=candidate_data8,
                                    index='Partei_ABK',
                                    columns='party_similarity',
                                    values=0,
                                    aggfunc='mean'
                                    ).round(decimals=2)

# computing the mean of diagonal elements
average_diagonal8 = np.trace(average_similarity8) / 7

# computing the mean of off-diagonal elements
average_off_diagonal8 = (average_similarity8.values.sum() - average_diagonal8 * 7) / 88

#the difference can be seen as a performance metric of the model 
average_diff8 = average_diagonal8 - average_off_diagonal8

In [507]:
import math

n_subplots = len(models_by_name)

plot_titles = list(models_by_name.keys())

fig2 = tools.make_subplots(rows=math.ceil(n_subplots/2), cols=2, subplot_titles=plot_titles)

row = 1
col = 1

count = 1

for name, trained_model in models_by_name.items():
    mask = [tag in candidate_data['from_name'].values for tag in trained_model.docvecs.offset2doctag]
    candidate_vecs = trained_model.docvecs.doctag_syn0[mask]
    
    X = candidate_vecs
    tsne = TSNE(n_components=2)
    X_tsne = tsne.fit_transform(X)
    
    trace = go.Scatter(x=X_tsne[:, 0], y=X_tsne[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
    fig2.append_trace(trace, row, col)
             
    if count % 2 == 0:
        row = row + 1
        col = 1
    else:
        col = col +1
        
    count += 1
    
fig2['layout'].update(title='Plots', showlegend=False, autosize=False, height=800, width=1000)
    
pyoff.iplot(fig2, filename='tsnePlot2')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [286]:
n_subplots = len(models_by_name)

cols = 2
d_array = np.array([{'is_3d': True}, {'is_3d': True}])
np.tile(d_array, (n_subplots/2, 1))

d3_array = np.arry([{'is_3d': True}])
np.repeat(3d_array, n_subplots)

fig3d = tools.make_subplots(rows=n_subplots//2, cols=2, 
                            specs=[[{'is_3d': True}, {'is_3d': True}]])

row = 1
col = 1

count = 1

for name, trained_model in models_by_name.items():
    mask = [tag in candidate_data['from_name'].values for tag in trained_model.docvecs.offset2doctag]
    candidate_vecs = trained_model.docvecs.doctag_syn0[mask]
    
    X = candidate_vecs
    tsne = TSNE(n_components=3)
    X_tsne = tsne.fit_transform(X)
    
    trace = go.Scatter3d(x=X_tsne[:, 0], y=X_tsne[:, 1], z=X_tsne[:, 2],
                       name=name,
                       mode='markers',
                       marker=dict(color=candidate_data['color'], 
                                   showscale=False,
                                   line=dict(color='black', width=1)),
                       text=candidate_data['from_name'])
    
    fig3d.append_trace(trace, row, col)
             
    if count % 2 == 0:
        row = row + 1
        col = 1
    else:
        col = col +1
        
    count += 1
    
fig3d['layout'].update(title='Plots', showlegend=False)
    
pyoff.iplot(fig3d, filename='tsnePlot2')

SyntaxError: invalid syntax (<ipython-input-286-b01d156a361a>, line 8)